In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
from datetime import datetime

import matplotlib.pyplot as plt
%matplotlib inline

### data load

In [2]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

print("")
print("train.num =", mnist.train.num_examples,
     ", test.num =", mnist.test.num_examples,
     ", validation.num =", mnist.validation.num_examples)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.

train.num = 55000 , test.num = 10000 , validation.num = 5000


### hyper parameter 

In [3]:
# Hyper-Parameter
learning_rate = 1e-3
epochs = 30
batch_size = 100

In [4]:
# 입력과 정답을 위한 플레이스홀더 정의
X = tf.placeholder(tf.float32, [None, 784])  
T = tf.placeholder(tf.float32, [None, 10])  

# 입력층의 출력 값. 컨볼루션 연산을 위해 reshape 시킴
A1 = X_img = tf.reshape(X, [-1, 28, 28, 1])  # image 28 x 28 x 1 (black / white)

### 컨볼루션층 1 5x5 크기의 128개 필터로 변경

In [5]:
# 1번째 컨볼루션 층, 5x5x128 필터 
W2 = tf.Variable(tf.random_normal([5, 5, 1, 128], stddev=0.01))  # standard deviation 표준편차 0.01 이내로 뽑음 -> 더 정교한 데이터
b2 = tf.Variable(tf.random_normal([128]))

# 1번째 컨볼루션 연산을 통해 28 x 28 x 1 => 28 x 28 x 128  흑백인 1개 층이 -> 128개 층을 거치게 됨
C2 = tf.nn.conv2d(A1, W2, strides=[1, 1, 1, 1], padding='SAME')

# relu
Z2 = tf.nn.relu(C2+b2)

# 1번째 max pooling을 통해 28 x 28 x 128 => 14 x 14 x 128  max pooling을 통해 4개를 1개로 묶어 가로 세로 2배씩 줄어들게 됨
A2 = P2 = tf.nn.max_pool(Z2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

### 완전연결층 

In [6]:
# 14 x 14 크기를 가진 32개의 activation map을 flatten 시킴
A2_flat = P2_flat = tf.reshape(A2, [-1, 14*14*128])  # 행렬로 변환, -1 : 행은 상관 없고, 14*14*32 : 열은 이거를 만족하게 만들어야 함

### 출력층

In [7]:
# 출력층
W3 = tf.Variable(tf.random_normal([14*14*128, 10], stddev=0.01))
b3 = tf.Variable(tf.random_normal([10]))

# 출력층 선형회귀 값 Z3, 즉 softmax 에 들어가는 입력 값
Z3 = logits = tf.matmul(A2_flat, W3) + b3

y = A3 = tf.nn.softmax(Z3)

In [8]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = Z3, labels = T))

optimizer = tf.train.AdamOptimizer(learning_rate)  # GradientDecent 방식을 사용하면 정답률이 91 정도로 떨어짐
## Adam(momentum)방식이 mnist 데이터에 잘 맞게 tf가 최적화 되어 있음.

train = optimizer.minimize(loss)

In [9]:
# batch_size x 10 데이터에 대해 argmax를 통해 행단위로 비교함
predicted_val = tf.equal(tf.argmax(A3, 1), tf.argmax(T, 1))

# batch_size x 10 의 True, False 를 1 또는 0 으로 변환
accuracy = tf.reduce_mean(tf.cast(predicted_val, dtype=tf.float32))

accuracy_index = tf.cast(predicted_val, dtype=tf.float32)

predicted_list = tf.argmax(A3, 1)

In [10]:
index_label_false_list_1 = []

with tf.Session() as sess:  # with를 쓰면 close 하지 않아도 됨
    sess.run(tf.global_variables_initializer())  # 변수 노드 (tf.Variable) 초기화
    
    start_time = datetime.now()
    
    for i in range(epochs):  # 30번 반복 수행
        total_batch = int(mnist.train.num_examples / batch_size)  # 55000 / 100
        
        for step in range(total_batch):
            batch_x_data, batch_t_data = mnist.train.next_batch(batch_size)
            
            loss_val, _ = sess.run([loss, train], feed_dict={X: batch_x_data, T: batch_t_data})    
        
            if step % 100 == 0:
                print("epochs = ", i, ", step = ", step, ", loss_val = ", loss_val)             
    
    end_time = datetime.now()
    
    print("")
    print("Elapsed Time => ", end_time-start_time)
    
    # Accuracy 확인
    test_x_data = mnist.test.images  # 10000 x 784
    test_t_data = mnist.test.labels  # 10000 x 10
    
    accuracy_val, predicted_list_val, index_label = sess.run([accuracy, predicted_list, accuracy_index], feed_dict={X: test_x_data, T: test_t_data})
    
    print("\nAccuracy =", accuracy_val)
    
    index_label_list = list(index_label)
    print("length of index_label_list = ", len(index_label_list))
    print("false label count = ", index_label_list.count([0]))
    
    # list type 으로 디버그
    temp_list = [] 
    
    for index in range(len(index_label_list)):
        
        if index_label_list[index] == 0:
            
            temp_list.append(index)
            temp_list.append(np.argmax(test_t_data[index]))  # one-hot encoding 이므로 argmax 로 정답 추출
            temp_list.append(predicted_list_val[index])
            
            index_label_false_list_1.append(temp_list)
            
            temp_list = []
            
    print("\nlength of index_label_false_list_1", len(index_label_false_list_1))

epochs =  0 , step =  0 , loss_val =  3.2188709
epochs =  0 , step =  100 , loss_val =  0.69338655
epochs =  0 , step =  200 , loss_val =  0.46189108
epochs =  0 , step =  300 , loss_val =  0.33188465
epochs =  0 , step =  400 , loss_val =  0.21028854
epochs =  0 , step =  500 , loss_val =  0.23892282
epochs =  1 , step =  0 , loss_val =  0.2903001
epochs =  1 , step =  100 , loss_val =  0.23478727
epochs =  1 , step =  200 , loss_val =  0.07630159
epochs =  1 , step =  300 , loss_val =  0.18351811
epochs =  1 , step =  400 , loss_val =  0.10681834
epochs =  1 , step =  500 , loss_val =  0.06879854
epochs =  2 , step =  0 , loss_val =  0.14692444
epochs =  2 , step =  100 , loss_val =  0.02326676
epochs =  2 , step =  200 , loss_val =  0.06530914
epochs =  2 , step =  300 , loss_val =  0.09805751
epochs =  2 , step =  400 , loss_val =  0.04555264
epochs =  2 , step =  500 , loss_val =  0.15371126
epochs =  3 , step =  0 , loss_val =  0.08188495
epochs =  3 , step =  100 , loss_val =  0

epochs =  26 , step =  100 , loss_val =  0.0010257057
epochs =  26 , step =  200 , loss_val =  0.0013052777
epochs =  26 , step =  300 , loss_val =  0.016100476
epochs =  26 , step =  400 , loss_val =  0.004486833
epochs =  26 , step =  500 , loss_val =  0.0014385837
epochs =  27 , step =  0 , loss_val =  0.0013394669
epochs =  27 , step =  100 , loss_val =  0.00016994038
epochs =  27 , step =  200 , loss_val =  0.00011387156
epochs =  27 , step =  300 , loss_val =  0.000535998
epochs =  27 , step =  400 , loss_val =  0.0009154407
epochs =  27 , step =  500 , loss_val =  0.00012017638
epochs =  28 , step =  0 , loss_val =  0.0012163682
epochs =  28 , step =  100 , loss_val =  0.0017730831
epochs =  28 , step =  200 , loss_val =  0.00048742007
epochs =  28 , step =  300 , loss_val =  0.0013117662
epochs =  28 , step =  400 , loss_val =  0.0005871617
epochs =  28 , step =  500 , loss_val =  0.0013702601
epochs =  29 , step =  0 , loss_val =  0.0020930727
epochs =  29 , step =  100 , loss

In [11]:
print(index_label_false_list_1)

[[211, 5, 3], [259, 6, 0], [321, 2, 7], [340, 5, 3], [582, 8, 2], [583, 2, 7], [659, 2, 1], [674, 5, 3], [684, 7, 3], [740, 4, 9], [839, 8, 3], [844, 8, 7], [947, 8, 9], [1014, 6, 5], [1039, 7, 3], [1112, 4, 6], [1226, 7, 2], [1232, 9, 4], [1242, 4, 9], [1247, 9, 3], [1260, 7, 1], [1296, 6, 5], [1319, 8, 0], [1326, 7, 2], [1393, 5, 3], [1414, 9, 3], [1530, 8, 7], [1553, 9, 3], [1621, 0, 6], [1641, 5, 6], [1709, 9, 3], [1717, 8, 0], [1737, 5, 3], [1754, 7, 2], [1790, 2, 7], [1878, 8, 3], [1901, 9, 8], [2004, 8, 9], [2035, 5, 3], [2098, 2, 0], [2118, 6, 0], [2129, 9, 8], [2130, 4, 9], [2135, 6, 1], [2189, 9, 1], [2224, 5, 3], [2293, 9, 4], [2329, 0, 2], [2369, 5, 3], [2406, 9, 4], [2514, 4, 9], [2582, 9, 3], [2597, 5, 3], [2648, 9, 5], [2654, 6, 1], [2758, 8, 3], [2760, 9, 4], [2778, 4, 0], [2810, 5, 3], [2896, 8, 0], [2930, 5, 7], [2939, 9, 5], [2970, 5, 3], [2979, 9, 0], [3030, 6, 0], [3060, 9, 3], [3384, 2, 6], [3422, 6, 0], [3441, 7, 8], [3503, 9, 1], [3520, 6, 4], [3558, 5, 0], [355